## Задание 

- Реализовать класс, моделирующий стохастическую среду Frozen Lake 8x8.
- Обучить агента Dyna-Q, использующего стохастическую модель среды и сравнить с простым Q-агентом.



Для реализации задачи сначала создадим класс для моделирования стохастической среды Frozen Lake 8x8. Затем реализуем агента Dyna-Q и классического Q-learning агента для сравнения их эффективности. После этого визуализируем графики обучения.

Импортируем все необходимые библиотеки.

In [1]:
import gymnasium as gym
import numpy as np
import random
import matplotlib.pyplot as plt

**Реализация Dyna-Q** 

In [2]:
class DynaQAgent:
    def __init__(self, env, alpha=0.3, gamma=0.99, epsilon=0.9, planning_steps=10):
        self.env = env
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.q_table = np.zeros((env.observation_space.n, env.action_space.n))
        self.model = {}
        self.planning_steps = planning_steps

    def choose_action(self, state):
        if random.uniform(0, 1) < self.epsilon:
            return self.env.action_space.sample()  # случайное действие
        else:
            return np.argmax(self.q_table[state])  # наилучшее действие по Q-таблице

    def learn(self, state, action, reward, next_state, done):
        # Обновляем Q-таблицу на основе реального опыта
        q_predict = self.q_table[state][action]
        q_target = reward + self.gamma * np.max(self.q_table[next_state]) * (1 - done)
        self.q_table[state][action] += self.alpha * (q_target - q_predict)

        # Обновляем модель среды
        self.model[(state, action)] = (reward, next_state, done)

        # Планируем шаги на основе модели
        for _ in range(self.planning_steps):
            s, a = random.choice(list(self.model.keys()))
            r, s_, d = self.model[(s, a)]
            q_predict_model = self.q_table[s][a]
            q_target_model = r + self.gamma * np.max(self.q_table[s_]) * (1 - d)
            self.q_table[s][a] += self.alpha * (q_target_model - q_predict_model)

**Реализация Q-learning**

In [3]:
class QLearningAgent:
    def __init__(self, env, alpha=0.3, gamma=0.99, epsilon=0.9):
        self.env = env
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.q_table = np.zeros((env.observation_space.n, env.action_space.n))

    def choose_action(self, state):
        if random.uniform(0, 1) < self.epsilon:
            return self.env.action_space.sample()  # случайное действие
        else:
            return np.argmax(self.q_table[state])  # наилучшее действие по Q-таблице

    def learn(self, state, action, reward, next_state, done):
        # Обновляем Q-таблицу на основе реального опыта
        q_predict = self.q_table[state][action]
        q_target = reward + self.gamma * np.max(self.q_table[next_state]) * (1 - done)
        self.q_table[state][action] += self.alpha * (q_target - q_predict)

**Реализация функции обучения агента.**

In [4]:
def train_agent_with_epsilon_decay(agent, episodes, epsilon_decay=0.99):
    rewards = []
    for episode in range(episodes):
        state = agent.env.reset()[0]
        total_reward = 0
        done = False
        while not done:
            action = agent.choose_action(state)
            next_state, reward, done, _, _ = agent.env.step(action)
            agent.learn(state, action, reward, next_state, done)
            state = next_state
            total_reward += reward
        rewards.append(total_reward)
        # Плавное уменьшение epsilon
        agent.epsilon *= epsilon_decay
    return rewards

**Создание среды FrozenLake 8x8**

In [5]:
env = gym.make("FrozenLake-v1", map_name="8x8", is_slippery=True)

**Создание агентов Dyna-Q и Q-learning**

In [6]:
dyna_q_agent = DynaQAgent(env, alpha=0.3, gamma=0.99, epsilon=0.9, planning_steps=10)
q_learning_agent = QLearningAgent(env, alpha=0.3, gamma=0.99, epsilon=0.9)

**Обучение агентов**

In [10]:
episodes = 10  # увеличиваем количество эпизодов для лучшего обучения
dyna_q_rewards = train_agent_with_epsilon_decay(dyna_q_agent, episodes)
q_learning_rewards = train_agent_with_epsilon_decay(q_learning_agent, episodes)

**Визуализация результатов**

In [ ]:
plt.plot(dyna_q_rewards, label="Dyna-Q")
plt.plot(q_learning_rewards, label="Q-Learning")
plt.xlabel("эпизоды")
plt.ylabel("вознаграждение")
plt.legend()
plt.title("Сравнение агентов")
plt.show()

## Вывод

Нами были изучены 2-а агента Dyna-Q и  Q-learning.  Испытания на своем железе показало не хватку ресурсов для корректного обучения и получания наивысшего вознаграждения и отображения его на графике. Обучение 100 эпиходов не дало нормальной картины на графике, для этого есть несколько причин- 

- Низкая вероятность успеха в среде
- не найдены оптимальные параметры обучения. 
- Начальные условия Q-таблицы

для того, что бы улучшить значения нужно:

- увеличить количество эпох. 
- подобрать наилучшие параметры обучения. 

В моих реалиях это не наилучший вариант. 